In [1]:
import sys 
sys.path.append('nngp')

import numpy as np
import tensorflow as tf

from sklearn.datasets import fetch_mldata
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer 
from sklearn.metrics import mean_squared_error

from nngp.nngp import NNGPKernel
from nngp.gpr import GaussianProcessRegression

print(tf.__version__)

1.5.0


In [2]:
mnist = fetch_mldata('MNIST original', data_home='mnist')
mnist.DESCR

'mldata.org dataset: mnist-original'

In [3]:
lb = LabelBinarizer()
lb.fit(mnist.target)
# encode target labels as zero-mean one hot encoded vector 
# with negative class = -0.1 and positive class as 0.9
encode = lambda y: lb.transform(y) - .1

In [4]:
image_train, image_test, label_train, label_test = train_test_split(
    mnist.data, mnist.target, stratify = mnist.target, random_state = 444, test_size=.15)

X_test = image_test
y_test = encode(label_test)

In [5]:
def get_train_subset(n):
    x, _, y, _ = train_test_split(image_train, label_train, stratify = label_train, random_state=333, train_size = n)
    return x, encode(y)

In [6]:
def train_or_load_model(sample_size):
    grid_path = 'grids/mnist_s%d' % sample_size
    nngp_kernel = NNGPKernel(
        depth = 8,
        weight_var=1.79,
        bias_var=0.83,
        nonlin_fn= tf.tanh,
        grid_path = grid_path,
        use_precomputed_grid = True,
        n_gauss=301,
        n_var=301,
        n_corr=301,
        max_gauss=10,
        max_var=100,
        use_fixed_point_norm=False)
    X_train, y_train = get_train_subset(sample_size)
    return GaussianProcessRegression(X_train, y_train, kern=nngp_kernel)

In [ ]:
def accuracy(y, y_hat):
    return np.mean(np.argmax(y, axis = 1) == np.argmax(y_hat, axis = 1))

In [ ]:
sample_sizes = np.array([1000, 5000, 10000, 25000, 50000])
nngp_results = np.empty((sample_sizes.shape[0], 2))

for i, sample_size in enumerate(sample_sizes):
    with tf.Session() as sess:
        m = train_or_load_model(sample_size = sample_size)
        y_hat, _ = m.predict(X_test, sess)
        nngp_results[i, 0] = accuracy(y_test, y_hat)  
        nngp_results[i, 1] = mean_squared_error(y_test, y_hat)
        print('done sample size %d' % (sample_size))
        print('accuarcy %.3f mean squared error %.3f' % (nngp_results[i, 0], nngp_results[i, 1]))

INFO:tensorflow:Generating interpolation grid...
